# Datenbankprogrammierung

**Was ist ein Cursor? Definieren Sie das Konzept in Ihren eigenen Worten.**

Bei einem Cursor handelt es sich um ein Objekt, welches das Iterieren über die einzelnen Tupeln (Zeilen) einer Tabelle ermöglicht.
  
**Aus welchem Grund (warum) und zu welchem Zweck (wozu) braucht man Cursors?**

Man brauch Cursors, weil grosse Tabellen nicht zuerst komplett ausgelesen und erst danach verarbeitet werden können. Sie erlauben einzelne Tupel schnell zu bearbeiten.

**Wozu werden Datenbanksprachen in andere Sprachen eingebettet?**

Damit die Businesslogik nicht in der Datenbank implementiert werden muss, sondern in der Sprach (oder Codebase) der eigentlichen Applikation.

**Was ist der Unterschied zwischen objektorientierten und objektrelationalen Datenbanken?**

Objektorientierte Datenbanken bestehen grundsätzlich nur aus Objekten (so wie bei Java auch fast alles ein Objekt ist). Objektrelationale Datenbanken sind relationale Datenbanken, welche um gewissen Eigenschaften von der OO erweitert wurden.

**Was ist ein Surrogat?, und was hat es mit Objektorientierung zu tun?**

Ein Surrogat ist ein Schlüssel welcher vom Datenbanksystem erstellt wird. Ein Surrogat wird einem Tupel zugewiesen und kann nicht verändert werden. Das Tupel lässt sich über sein Surrogat identifizieren.
Eine Tabelle kann das Surrogat einer zweiten Tabelle als Fremdschlüssel referenzieren um eine logische Abhängigkeit der Tupeln darzustellen.
Kurz: Ein Surrogat ist ein autogenerierter Primarykey.
Das Surrogat erlaubt es, Aggregationen darzustellen: Tupel A ist Teil von Tupel B.
Mit "PART_OF(Tabelle_XY)" können solche Beziehungen der Datenbank beigebracht werden. Die Logischen Tabellen werden dabei beibehalten, physisch werden sie jedoch zu einem Objekt zusammengefasst.

**Was ist das NF2 Modell? Was ist der Zusammenhang von NF2 mit der Objektorientierung?**

Non First Normal Form", Tupeln können ganze Tabellen als Eigenschaften haben. Zu vergleichen mit einem Objekt A, welches eine Liste von Objekten B als Eigenschaft hat.

**Was ist objekt-relationales Mapping, und was sind die Vorteile?**

Ein OR-Mapper ist eine Softwarekomponente, welche relationale Tabellen in die objektorientierte Welt abbildet.
Es entfallen die hohen Kosten und Risiken einer Migration von relationaler- zu objektorientierter Datenbank. 
Bewährte Technologien und Mindsets aus der SQL Welt können mit jenen der OO Welt kombiniert werden.


## Jupyter Setup

Um dieses Jupyter Notebook zu gebrauchen sollten folgende extensions
installiert werden:

- https://github.com/SpencerPark/IJava


## Datenbank Setup

Als erstes sollte [mysql installiert sein](https://support.rackspace.com/how-to/install-mysql-server-on-the-ubuntu-operating-system/).

Die Datenbank kann folgendermassen aufgesetzt werden:

```sh
sudo mysql < uni-schema.sql
sudo mysql uni4 < uni-daten.sql
```

## Hello JDBC

Professoren und Vorlesungen selektieren (ohne join)

In [1]:
%maven mysql:mysql-connector-java:8.0.19

import java.sql.Statement;
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;

In [2]:
Connection connection = DriverManager.getConnection(
    "jdbc:mysql://localhost:3306/uni4?&serverTimezone=Europe/Zurich",
    "uniuser", 
    "password"
);

In [3]:
var professorResultset = connection.createStatement().executeQuery("SELECT * FROM Professoren");
while (professorResultset.next()) {
    System.out.println(professorResultset.getString(2));
}

Precht
Sokrates
Russel
Kopernikus
Popper
Augustinus
Curie
Kant


Erweitern Sie dieses JDBC-Pogramm, so dass es die Namen aller Vorlesungen zu einem
Professor darstellt. Benutzen Sie dazu keinen Join, sondern die while-Schliefe innerhalb von
Java

In [4]:
// Professoren und Vorlesungen selektieren

var professorResultset = connection.createStatement().executeQuery("SELECT * FROM Professoren");
while (professorResultset.next()) {
    var statement = connection.prepareStatement("SELECT * FROM Vorlesungen WHERE gelesenVon = ?");
    statement.setInt(1, professorResultset.getInt(1));
    var vorlesungResult = statement.executeQuery();
    
    System.out.printf("%s unterrichtet: ", professorResultset.getString(2));
    while (vorlesungResult.next()) {
        System.out.printf("%s, ", vorlesungResult.getString(2));
    }
    System.out.println();
}

Precht unterrichtet: Postmoderne, 
Sokrates unterrichtet: Logik, Ethik, Maeeutik, 
Russel unterrichtet: Erkenntnistheorie, Wissenschaftstheorie, Bioethik, 
Kopernikus unterrichtet: 
Popper unterrichtet: Der Wiener Kreis, 
Augustinus unterrichtet: Glaube und Wissen, 
Curie unterrichtet: 
Kant unterrichtet: Die 3 Kritiken, Grundzuege, 


## Exception Handling

Bringen Sie im Java-Projekt Hello JDBC die Java Klasse ExceptionExample zum laufen. 

Dazu können Sie die Stored Procedure testException auf dem MySQL Server erstellen. Die
Definition der entsprechenden Stored Procedure testException finden Sie im Kommentar der
Java Klasse.

Ziel: in Java wird eine SQLException geworfen

In [9]:
connection.createStatement().execute(
    "CREATE PROCEDURE testException ()" +
    "BEGIN" +
    "    SIGNAL SQLSTATE '45000'" +
    "    SET MESSAGE_TEXT = 'An Error occurred', " +
    "    MYSQL_ERRNO = 1001;" +
    "END"
);

EvalException: PROCEDURE testException already exists

In [5]:
var resultset = connection.createStatement().executeQuery("CALL testException()");

EvalException: An Error occurred

## Stored Procedures

Eine Stored-Procedure erlaubt es innerhalb der Datenbank, auch Schleifen und
Fallunterscheidungen auf Resultatmengen von SQL-Anfragen direkt auf dem Datenbankserver zu
verwenden. Somit können auch Turing-vollständige Programme mit Iteration und Rekursion auf
der Datenbank ausgeführt werden. Hier sehen Sie exemplarisch anhand einer PL/PGSQL StoredProcedure, wie das funktioniert.

```sql
-- Erstellen Sie auf Ihrer lokalen MySQL-Installation eine Tabelle:
CREATE TABLE IF NOT EXISTS weekends(datum date primary key, yay boolean);

-- Dann erstellen Sie eine Stored Procedure `f`:
DELIMITER $$

CREATE PROCEDURE mark_weekends (startdate date, nr int) BEGIN

  DECLARE i int DEFAULT 1;
  DECLARE next_day date DEFAULT startdate;

  WHILE (nr >= i) DO

    -- insert weekdays and mark them as weekends
    INSERT INTO weekends(datum, yay)
    VALUES (next_day, (dayofweek(next_day) in (7, 1)));

    SET i := i + 1;
    SET next_day := DATE_ADD(next_day, INTERVAL 1 DAY);

  END WHILE;
END$$

DELIMITER ;

-- Anschliessend können Sie die die Prozedur wie folgt aufrufen:
CALL mark_weekends('2020-11-11', 111);
```

Mit dem Befehl ausführen:

```sh
mysql -u uniuser -p"password" uni4 < exercises/stored-procedures.sql
```

---

**Welche Semantik hat die Prozedur f (was genau macht das Programm)? Was bedeuten die Daten, welche die Prozedur generiert?**

Die Prozedur markiert alle Samstage und Sonntage (alle wochenenden)

---

Erstellen Sie ein Java-Programm, welches folgende Aktionen durchführt:

- Eingabe zweier Parameter via `main(String[] args)`: Anfangsdatum und Anzahl Tage
- Verbindungsaufnahme mit ihrem lokalen PostgreSQL Server
- Löschen der Tabelle t mit delete
- Aufruf der oben definierten Stored-Function f mit den eingegebenen Parametern
- Ausgabe des Inhalts der Kalender-Tabelle mit System.out.println()

Für den Aufruf von SQL-Statements, die kein Resultset zurückgeben, muss die Methode
`Statement.execute()` verwendet werden.

In [6]:
connection.createStatement().execute("DELETE FROM weekends");
connection.createStatement().execute("CALL mark_weekends('2020-11-11', 111)");

false

In [7]:
var result = connection.createStatement().executeQuery("SELECT * FROM weekends");
while (result.next()) {
    System.out.printf("%s: %d\n", result.getString(1), result.getInt(2));
}

2020-11-11: 0
2020-11-12: 0
2020-11-13: 0
2020-11-14: 1
2020-11-15: 1
2020-11-16: 0
2020-11-17: 0
2020-11-18: 0
2020-11-19: 0
2020-11-20: 0
2020-11-21: 1
2020-11-22: 1
2020-11-23: 0
2020-11-24: 0
2020-11-25: 0
2020-11-26: 0
2020-11-27: 0
2020-11-28: 1
2020-11-29: 1
2020-11-30: 0
2020-12-01: 0
2020-12-02: 0
2020-12-03: 0
2020-12-04: 0
2020-12-05: 1
2020-12-06: 1
2020-12-07: 0
2020-12-08: 0
2020-12-09: 0
2020-12-10: 0
2020-12-11: 0
2020-12-12: 1
2020-12-13: 1
2020-12-14: 0
2020-12-15: 0
2020-12-16: 0
2020-12-17: 0
2020-12-18: 0
2020-12-19: 1
2020-12-20: 1
2020-12-21: 0
2020-12-22: 0
2020-12-23: 0
2020-12-24: 0
2020-12-25: 0
2020-12-26: 1
2020-12-27: 1
2020-12-28: 0
2020-12-29: 0
2020-12-30: 0
2020-12-31: 0
2021-01-01: 0
2021-01-02: 1
2021-01-03: 1
2021-01-04: 0
2021-01-05: 0
2021-01-06: 0
2021-01-07: 0
2021-01-08: 0
2021-01-09: 1
2021-01-10: 1
2021-01-11: 0
2021-01-12: 0
2021-01-13: 0
2021-01-14: 0
2021-01-15: 0
2021-01-16: 1
2021-01-17: 1
2021-01-18: 0
2021-01-19: 0
2021-01-20: 0
2021-0

## Median Semester


Der Median ist der Wert eines Attributs, für den 50% der Datensätze einen kleineren oder
gleichen Wert haben, und 50% der Datensätze einen gleichen oder grösseren Wert aufweisen. In 
einer sortierten Tabelle ist es der Wert in der Mitte bei ungerader Anzahl Tupel und der
Durchschnitt der beiden mittleren Tupel be gerader Anzahl.

Programmieren Sie in MySQL eine stored Function, welche den Median der Anzahl Semester
der Studenten berechnet:

```sql
DROP PROCEDURE IF EXISTS median_semester;

DELIMITER $$

CREATE PROCEDURE median_semester () BEGIN

  -- https://stackoverflow.com/a/7263925
  SELECT AVG(dd.Semester) as median_semester
  FROM (
    SELECT s.Semester, @rownum:=@rownum+1 as row_number, @total_rows:=@rownum
      FROM Studenten s, (SELECT @rownum:=0) r
      WHERE s.Semester is NOT NULL
      ORDER BY s.Semester
  ) as dd
  WHERE dd.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) );

END$$

DELIMITER ;

CALL median_semester();
```

Mit dem Befehl ausführen:

```sh
mysql -u uniuser -p"password" uni4 < exercises/studenten-median.sql
```

## JPA Coding

Bringen Sie das Beispiel-Projekt HelloJPA zum laufen:

```sh
ant compile jar
java -jar ./dist/lib/h2/bin/h2-1.4.200.jar
```

Die H2 Konsole wird im Browser geöffnet. Verwenden Sie folgendes Login:
 - URL: jdbc:h2:./dist/lib/h2/data/uni
 - User: u, Password: p
 
 
Feedback zur abgegebenen Java solution:

- Binarys eingecheckt (wieso nicht maven/gradle)
- Persistance XML mit passwort (sollte nun vielleicht geändert werden)
- Persistance provider nicht mehr kompatibel mit java 11+ `implementation'org.hibernate:hibernate-core:5.3.1.Final'`

---

**Schauen Sie sich die Datei persistence.xml an. Wie heisst die Persistence Unit? Wie lautet die URL zur H2 Datenbank?**

- Unit Name: `HelloJpaPU2`
- URL: `jdbc:h2:./dist/lib/h2/data/uni` bzw `jdbc:mysql://localhost:3306/uni4?useSSL=false&amp;serverTimezone=UTC`


**Schauen Sie sich die Datei Professoren.java an. Wie bzw. mit welchen Annotationen wurden die Primär- und Fremdschlüssel spezifiziert?**

Ein Primary key wird folgendermassen gekennzeichnet:

```java
@Id
@Basic(optional = false)
@Column(name = "id")
private Integer primaryKey;
```

Beziehungen zu anderen Entitaeten werden per 'Collections' abgebildet:

```java
@OneToMany(mappedBy = "field_name")
private Collection<Entity> relationShip;
```

**Schauen Sie die generierte Datei Studenten.java an. Mit welchen Annotationen wurde die many-to-many-Relationship umgesetzt? Wie sieht es diesbezüglich bei Vorlesungen.java aus?**

Vorlesungen:

```java
@ManyToMany(mappedBy = "vorlesungenCollection")
private Collection<Studenten> studentenCollection;
```

Studenten:

```java
@JoinTable(name = "h\u00f6ren", joinColumns = {
    @JoinColumn(name = "MatrNr", referencedColumnName = "MatrNr")}, inverseJoinColumns = {
    @JoinColumn(name = "VorlNr", referencedColumnName = "VorlNr")})

@ManyToMany
private Collection<Vorlesungen> vorlesungenCollection;
```


**Wie wurde NOT NULL in JPA-Annotationen umgesetzt?**
 
Mit `@Basic(optional = false)`
 
---


- Instanzieren Sie einen Entity-Manager mit Variablenname em
- Geben Sie für alle Professoren den Namen und seine Vorlesungen aus via `System.out`. Verwenden Sie dazu die generierte NamedQuery Professoren.findAll mit der Methode `em.createNamedQuery`.
- Selektieren Sie via JPQL die Vorlesungen, welche Sokrates liest, und geben sie Sie via `System.out` aus. Verwenden sie dazu eine neue JPQL Query mit `em.createQuery`.
- Setzen sie den Raum von Sokrates auf 1234 und persistieren sie dies auf der Datenbank mit `em.merge()`. Dazu müssen Sie eine Transaktion mit `ta=em.getTransaction()` instanzieren, diese vor dem merge beginnen mit `ta.begin()` und nach dem merge committen mit `ta.commit()`.
- Fügen Sie einen neuen Professor mit Namen „Precht“ via JPA hinzu mit `em.persist()`
- Fügen Sie eine neue Vorlesung mit Namen „Postmoderne“ hinzu, welche Precht liest. 


```java

/* Instanzieren Sie einen Entity-Manager mit Variablenname em */
final EntityManagerFactory emf = Persistence.createEntityManagerFactory("HelloJpaPU2");
final EntityManager em = emf.createEntityManager();


final Query query = em.createNamedQuery("Professoren.findAll");
for (final Professoren p : (List<Professoren>) query.getResultList()) {
    System.out.println(p.getName());

    for (final Vorlesungen v : p.getVorlesungenCollection()) {
        System.out.printf("  - %s\n", v.getTitel());
    }
}


/* Selektieren Sie via JPQL die Vorlesungen, welche Sokrates liest, und geben sie Sie via
 System.out aus. Verwenden sie dazu eine neue JPQL Query mit em.createQuery. */
 System.out.println("\nSokrates liest folgende vorlesungen:");
 em.createQuery("SELECT v FROM Vorlesungen v, Professoren p WHERE p.name = :profName and v.gelesenVon = p.persNr")
    .setParameter("profName", "Sokrates")
    .getResultList()
    .forEach((v) -> System.out.printf("  - %s\n", ((Vorlesungen)v).getTitel()));

em.getTransaction().begin();

/* Setzen sie den Raum von Sokrates auf 1234 und persistieren sie dies auf der Datenbank mit em.merge() */
System.out.println("\nSokrates hat nun folgendn raum:");
final Professoren sokrates = (Professoren)em.createNamedQuery("Professoren.getByName")
    .setParameter("name", "Sokrates")
    .getResultList().get(0);

sokrates.setRaum(1234);
em.merge(sokrates);

System.out.println(((Professoren)em.createNamedQuery("Professoren.getByName")
    .setParameter("name", "Sokrates")
    .getResultList().get(0)).getRaum());

/* Fügen Sie einen neuen Professor mit Namen „Precht“ via JPA hinzu (em.persist()) */
System.out.println("\nNun arbetet auch Precht hier!");
final Professoren precht = new Professoren(1337, "Precht");
em.persist(precht);

System.out.println(((Professoren)em.createNamedQuery("Professoren.getByName")
    .setParameter("name", "Precht")
    .getResultList().get(0)).getName());

/* Fügen Sie eine neue Vorlesung mit Namen „Postmoderne“ hinzu, welche Precht liest */
System.out.println("\nPrecht ist nicht faul, er macht Postmoderne!");
final Vorlesungen postmoderne = new Vorlesungen(2369);
postmoderne.setTitel("Postmoderne");
postmoderne.setGelesenVon(precht);
em.persist(postmoderne);

em.getTransaction().commit();
```

Ausfuehren mit `gradle run`

---

## Datenbankprogrammierung NoSQL

```python
# connect
db = MongoClient('mongodb://localhost:27017/').test_database

# insert data:
post = {
    "type": "text",
    "content": {
        "title": "An interesting post",
        "content": "Hmmm",
    },
    "timestamp": datetime.now(),
    "feed_owner": "Geralt"
}

post_id = db.posts.insert_one(post).inserted_id

# query data:
posts_query = (db.posts
    # get todays images of the person with 'name'
    .find({
        "$and": [{
            "timestamp": {
                "$gte": start,
                "$lt": end
            },
            "feed_owner": "Geralt"
        }]
    })
    
```

Stored Procedures

- http://dirolf.com/2010/04/05/stored-javascript-in-mongodb-and-pymongo.html
